# Afterpulse Detection (dead-time / internal reflections)
To figure out how to detect and remove afterpulses, and plot examples.
- use photon rate just around the lake surface elevation (depending on strong/weak beam??)
- then use peaks distances that are commonly found

### From the literature
afterpulses at: ∼0.45, ∼0.9, ∼2.3, and ∼4.2 m

The afterpulses captured from on-orbit measurements are caused by three different reasons: (1) the effects of the dead-time circuit (∼3 ns) due to PMT saturation; (2) the effects of optical reflections within the ATLAS receiver optical components; (3) PMT afterpulses. The echoes separated by ∼0.45 m are attributed to the effect of the dead-time circuit (∼3 ns) due to PMT saturation. The echoes at ∼2.3 and ∼4.2 m below the primary surface returns are caused by the optical reflections within the ATLAS receiver optical components, while the echoes from ∼10 to ∼45 m away from the primary surface signal are due to the PMT afterpulses with a longer time delay.

Lu, X., Hu, Y., Yang, Y., Vaughan, M., Palm, S., Trepte, C., ... & Baize, R. (2021). Enabling value added scientific applications of ICESat‐2 data with effective removal of afterpulses. Earth and Space Science, 8(6), e2021EA001729.

In [1]:
%matplotlib widget
import pickle
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import Image, display
from cmcrameri import cm as cmc
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.rcParams.update({'font.size': 6})

In [ ]:
# lake 3 (gt1l, weak)
# lake 13* (gt1r, strong)
# lake 23 (gt1r, strong)
# lake 24 (gt1r, strong)
# lake 28 (gt1r, strong)
# lake 31** (gt2l, weak)
# lake 35** (gt2l, weak)
# lake 36** (gt2l, weak)
# lake 37** (gt2l, weak)
# lake 39*** (gt2l, weak)
# lake 43 (gt2r, strong)
# lake 44* (gt2r, strong)
# lake 45 (gt2r, strong)
# lake 46** (gt2r, strong)
# lake 48 (gt2r, strong)
# lake 50*** (gt2r, strong)
# lake 51** (gt2r, strong)
# lake 53** (gt2r, strong)
# lake 54 (gt2r, strong)

In [ ]:
def get_surface_extent(self, surf_width=0.4, abov_width=2.0, bin_width=1.0, smooth=31, max_ratio=0.3, min_length=100.0):
    if smooth%2 == 1: smooth += 1
    surf_selector = (self.photon_data.h > (self.surface_elevation-surf_width/2)) & (self.photon_data.h < (self.surface_elevation+surf_width/2))
    abov_selector = (self.photon_data.h > (self.surface_elevation+surf_width/2)) & (self.photon_data.h < (self.surface_elevation+surf_width/2+abov_width))
    totl_selector = (self.photon_data.h < (self.surface_elevation-surf_width/2)) | (self.photon_data.h > (self.surface_elevation+surf_width/2))
    df_surf = self.photon_data[surf_selector]
    df_abov = self.photon_data[abov_selector]
    df_totl = self.photon_data[totl_selector]
    bins = np.arange(start=self.photon_data.xatc.min(), stop=self.photon_data.xatc.max(), step=bin_width)
    mids = bins[:-1] + 0.5 * bin_width
    hist_surf = np.histogram(df_surf.xatc, bins=bins)
    hist_abov = np.histogram(df_abov.xatc, bins=bins)
    hist_totl = np.histogram(df_totl.xatc, bins=bins)
    max_all = binned_statistic(self.photon_data.xatc, self.photon_data.h, statistic='max', bins=bins)
    min_all = binned_statistic(self.photon_data.xatc, self.photon_data.h, statistic='min', bins=bins)
    surf_smooth = np.array(pd.Series(hist_surf[0]).rolling(smooth,center=True,min_periods=1).mean())
    abov_smooth = np.array(pd.Series(hist_abov[0]).rolling(smooth,center=True,min_periods=1).mean())
    totl_smooth = np.array(pd.Series(hist_totl[0]).rolling(smooth,center=True,min_periods=1).mean())
    maxs_smooth = np.array(pd.Series(max_all[0]).rolling(smooth,center=True,min_periods=1).max())
    mins_smooth = np.array(pd.Series(min_all[0]).rolling(smooth,center=True,min_periods=1).min())
    dens_surf = surf_smooth / (surf_width*bin_width)
    dens_abov = abov_smooth / (abov_width*bin_width)
    dens_totl = totl_smooth / ((maxs_smooth-mins_smooth-surf_width)*bin_width)
    dens_surf[dens_surf == 0] = 1e-20
    dens_ratio_abov = dens_abov / dens_surf
    dens_ratio_totl = dens_totl / dens_surf
    dens_ratio_abov[dens_ratio_abov>1] = 1
    dens_ratio_totl[dens_ratio_totl>1] = 1
    dens_eval = np.max(np.vstack((dens_ratio_abov,dens_ratio_totl)), axis=0)
    surf_possible = dens_eval < max_ratio
    surf_possible[(mids<250) | (mids>(np.max(mids)-250))] = False # because we added two extra major frames on each side

    # get surface segments that are continuous for longer than x meters
    current_list = []
    surface_segs = []
    i = 0
    while i < len(surf_possible):
        if surf_possible[i]:
            current_list.append(i)
        else:
            if (len(current_list) * bin_width) > min_length:
                surface_segs.append([mids[current_list[0]], mids[current_list[-1]]])
            current_list = []
        i += 1 
    self.surface_extent_detection = surface_segs

In [33]:
i = 50
fn = 'pickles/specular%02i.pkl' % i
with open(fn, 'rb') as f:
    lk = pickle.load(f)
    
plt.close('all')

thresh_upper = 1.0
thresh_lower = -5.5
top_width = 0.3
bin_h=0.01
smooth_h=0.1
smooth_pulse=11
if smooth_pulse%2 == 0: smooth_pulse += 1
max_ratio=0.3
min_length=100.0
extent_buffer = 10.0

col_top = 'black'
col_pk1 = 'red'
col_pk2 = 'blue'
col_pk3 = 'magenta'
col_pk4 = 'cyan'

surf_elev = lk['surface_elevation']
surf_ext = lk['surface_extent_detection']
df = lk['photon_data']
dfs = df[(df.h < (thresh_upper+surf_elev)) & (df.h > (thresh_lower+surf_elev))].copy()
dfs['h'] = dfs.h - surf_elev
is_in_extent = np.full(len(dfs), False, dtype=np.bool_)
for ext in surf_ext:
    is_in_extent[(dfs.xatc > (ext[0]-extent_buffer)) & (dfs.xatc < (ext[1]+extent_buffer))] = True
dfs = dfs[is_in_extent].copy()
# add a unique id for each pulse, to group by 
dfs['pulseid'] = dfs.apply(lambda row: 1000*row.mframe+row.ph_id_pulse, axis=1)

df_top = dfs[(dfs.h > (-top_width)) & (dfs.h < (top_width))]

# binning for peaks
bins = np.arange(start=thresh_lower, stop=thresh_upper, step=bin_h)
mids = bins[:-1] + 0.5 * bin_h
hist_h = np.histogram(dfs.h, bins=bins)
smooth = int(smooth_h/bin_h)
if smooth %2 == 0: smooth += 1
hist_h_smooth = np.array(pd.Series(hist_h[0]).rolling(smooth,center=True,min_periods=1).mean())
hist_h_plot = np.array(hist_h[0]) / hist_h_smooth.max()
hist_h_smooth /= hist_h_smooth.max()

def group_by_pulse(df_in,smoothing):
    thegroup = df_in.groupby('pulseid')
    df_grouped = thegroup[['xatc', 'lat', 'lon']].mean()
    df_grouped['ph_count'] = thegroup['h'].count()
    df_grouped['ph_count_smooth'] = np.array(df_grouped.ph_count.rolling(smoothing,center=True,min_periods=1).mean())
    return df_grouped

# looking at photon counts 
df_pulse = group_by_pulse(dfs,smooth_pulse)
df_pulse_top = group_by_pulse(df_top,smooth_pulse)

#__________________________________________________________________
# make the figure
fig, ((ax,ax2),(ax3,ax4)) = plt.subplots(nrows=2, ncols=2, figsize=[9, 5], dpi=100)
xlms = (surf_ext[0][0], surf_ext[-1][1])
ylms = (thresh_lower, thresh_upper)

#-------------------------------------------------
# selected photons
pscatt = ax.scatter(dfs.xatc, dfs.h, s=15, c='k', alpha=0.1, edgecolors='none',label='ATL03 data')
ax.scatter(dfs.xatc, dfs.h, s=0.1, c='r', alpha=0.2, edgecolors='none')
ax.plot(xlms, [0]*2, 'b-', lw=0.5)

# title and labels
ax.set_xlabel('along-track distance [m]')
ax.set_ylabel('elevation from lake surface [m]')
ax.set_xlim(xlms)
ax.set_ylim(ylms)

#-------------------------------------------------
# histogram showing peaks for specular returns
xlim_ax2 = (1e-4,10)
ax2.scatter(hist_h_plot, mids, s=3, color=(0.75, 0.75, 0.75), lw=0.5, edgecolors='none')
ax2.plot(hist_h_smooth, mids, 'k-')
ax2.plot(xlim_ax2, [0]*2, 'b-', lw=0.5)
ax2.set_xlabel('normalized photon counts')
ax2.axes.yaxis.set_ticklabels([])
ax2.set_xlim(xlim_ax2)
ax2.set_ylim(ylms)
ax2.set_xscale('log')

#-------------------------------------------------
# plot photon counts per shot
alph = 0.1
thisax = ax3
def plot_counts(df_in, col):
    thisax.scatter(df_in.xatc, df_in.ph_count, s=3, color=col, edgecolors='none', alpha=alph)
    thisax.plot(df_in.xatc, df_in.ph_count_smooth, color=col)
    
plot_counts(df_pulse, col='cyan')
plot_counts(df_pulse_top, col='magenta')

# thiscolor = 'black' # black
# ax3.scatter(df_pulse.xatc, df_pulse.ph_count, s=3, color=thiscolor, edgecolors='none', alpha=alph)
# ax3.plot(df_pulse.xatc, df_pulse.ph_count_smooth, color=thiscolor)

# thiscolor = 'red' # red
# ax3.scatter(df_pulse_top.xatc, df_pulse_top.ph_count, s=3, color=thiscolor, edgecolors='none', alpha=alph)
# ax3.plot(df_pulse_top.xatc, df_pulse_top.ph_count_smooth, color=thiscolor)
ax3.set_xlim(xlms)
         
#-------------------------------------------------
# the full lake, for reference
ax4.scatter(df.xatc, df.h, s=6, c=df.snr, alpha=0.3, edgecolors='none', cmap=cmc.lajolla, vmin=0, vmax=1)
rng = np.abs(surf_elev-np.min(lk['detection_2nd_returns']['h']))
ax4.set_xlim((df.xatc.min(),df.xatc.max()))
ax4.set_ylim((surf_elev-2*rng, surf_elev+rng))

fig.suptitle('example of specular return afterpulses over a melt lake in ICESat-2 data', y=0.98, fontsize=10)
fig.tight_layout()

#figname = 'figs_afterpulses/myfig.jpg'
# fig.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
pulse_group = dfs.groupby('pulseid')
df_pulse = pulse_group[['xatc', 'lat', 'lon']].mean()
df_pulse['ph_count'] = pulse_group['h'].count()
df_pulse['ph_count_smooth'] = np.array(df_pulse.ph_count.rolling(smooth_pulse,center=True,min_periods=1).mean())

pulse_group_top = df_top.groupby('pulseid')
df_pulse_top = pulse_group_top[['xatc', 'lat', 'lon']].mean()
df_pulse_top['ph_count'] = pulse_group_top['h'].count()
df_pulse_top['ph_count_smooth'] = np.array(df_pulse_top.ph_count.rolling(smooth_pulse,center=True,min_periods=1).mean())


In [24]:
df_pulse

,xatc,lat,lon,ph_count,ph_count_smooth
pulseid,,,,,
1573424354137,380.906611,67.235373,-48.994160,4,4.500000
1573424354138,381.612799,67.235379,-48.994162,6,4.428571
1573424354139,382.321260,67.235385,-48.994164,8,4.500000
1573424354140,383.032008,67.235392,-48.994166,3,4.333333
1573424354141,383.739420,67.235398,-48.994168,4,4.400000
...,...,...,...,...,...
1573424368020,2285.642376,67.252339,-48.999226,2,4.000000
1573424368021,2286.347855,67.252345,-48.999228,4,4.000000
1573424368022,2287.053004,67.252351,-48.999229,3,3.375000


In [ ]:
mframe_group = df.groupby('mframe')
    df_mframe = mframe_group[['lat','lon', 'xatc', 'dt']].mean()
    df_mframe.drop(df_mframe.head(1).index,inplace=True)
    df_mframe.drop(df_mframe.tail(1).index,inplace=True)
    df_mframe['time'] = df_mframe['dt'].map(convert_time_to_string)
    df_mframe['xatc_min'] = mframe_group['xatc'].min()
    df_mframe['xatc_max'] = mframe_group['xatc'].max()
    df_mframe['n_phot'] = mframe_group['h'].count()

In [ ]:
dfs

In [ ]:
lk['photon_data']

In [ ]:
ax.get_xlim()